In [8]:
import cv2
import numpy as np

def is_flip_successful(video_path):

  cap = cv2.VideoCapture(video_path)

  blue_lower = (70, 115, 55)
  blue_upper = (123, 200, 170)
  brown_lower = (0, 10, 40)  # Adjust as needed
  brown_upper = (70, 100, 135)
  min_cap_area = 1
  min_distance_threshold = 85

  while True:
    ret, frame = cap.read()
    if not ret:
      break

    frame = cv2.resize(frame, (640, 480))
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    height = frame.shape[0]

    roi_mask = np.zeros_like(frame, dtype=np.uint8)[:, :, 0]
    brown_mask = cv2.inRange(hsv_frame, brown_lower, brown_upper)

    roi_mask[int(height * 0.4):, :] = 255
    brown_mask = cv2.bitwise_and(brown_mask, roi_mask)
    blue_mask = cv2.inRange(hsv_frame, blue_lower, blue_upper)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kernel)
    brown_mask = cv2.morphologyEx(brown_mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cap_contour = None

    for cnt in contours:
      area = cv2.contourArea(cnt)
      if area > min_cap_area:
        cap_contour = cnt
        break

    if cap_contour is not None:
      M = cv2.moments(cap_contour)
      cX = int(M["m10"] / M["m00"])
      cY = int(M["m01"] / M["m00"])

      non_zero_pixels = cv2.findNonZero(brown_mask)
      closest_distance = float('inf')
      closest_pixel = None
      if non_zero_pixels is not None:  
        closest_distance = float('inf')
        closest_pixel = None
        for pixel in non_zero_pixels:
           distance = np.linalg.norm(np.array([cX, cY]) - pixel)
           if distance < closest_distance:
             closest_distance = distance
             closest_pixel = pixel
      else:
        print("No brown pixels found in this frame")        

      cv2.drawContours(frame, [cap_contour], 0, (0, 255, 0), 2)
      if closest_pixel is not None:
        cv2.circle(frame, tuple(closest_pixel[0]), 5, (0, 0, 255), -1)
        text = f"Distance: {closest_distance:.2f} px"
        cv2.putText(frame, text, (cX + 10, cY - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 255, 255), 1)

      is_flipped = closest_distance > min_distance_threshold
      if is_flipped:
        cv2.putText(frame, "Flipped!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
      else:
        cv2.putText(frame, "Not Flipped (on table)", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Bottle Flip Analysis", frame)
    if cv2.waitKey(100) == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

  return True

if __name__ == "__main__":
  video_path = "C:/Users/Shreyas/Downloads/flipping_assignment/no_flip_case_recording.mp4"
  is_flip_successful(video_path)
